In [1]:
import pandas as pd
import pathlib

## Calificar examen estudiantes: Dimensión 7

In [2]:
ruta_git = pathlib.Path().cwd().parent.parent.parent

In [3]:
est = pd.read_excel(ruta_git/'data/descargables/EstudiantesCFK.xlsx')

print(est.shape)
est.columns

(22982, 74)


Index(['N registro', 'Deseo participar en el estudio', 'Código IE', 'Grupo',
       'Nombre', 'Fecha', 'ID', 'Número de lista', 'Edad', 'Sexo',
       'Sector vivienda', 'Internet', 'Uso del dispositivo móvil',
       'Nivel escolaridad madre', 'Nivel escolaridad padre', 'Ocupación madre',
       'Ocupación padre', '¿Con quién vives?', 'Grado', '1.1. Ingeniería',
       '1.2 Matemáticas', '1.3 Educación', '1.4 Medicina',
       '1.5 Diseño gráfico', '1.6 Química', '1.7 Enfermería',
       '1.8 Desarrollo de software',
       '2.1 Soy capaz de sacar buenas notas en esta asignatura',
       '2.2 Si me va bien en esta asignatura, me ayudará en mi futura ocupación',
       '2.3 A mis padres les gustaría que eligiera un futuro profesional relacionado a esta asignatura',
       '2.4 Sé de alguien en mi familia que utiliza conocimientos relacionados a esta asignatura en su ocupación',
       'Comentarios 1-2', 'Un algoritmo es:',
       '¿Para qué sirven los algoritmos?', 'Un bucle es:',
    

In [4]:
est[est['Código IE']==65]

,N registro,Deseo participar en el estudio,Código IE,Grupo,Nombre,Fecha,ID,Número de lista,Edad,Sexo,...,4.3 La acumulación de basura procedente de las ciudades es un problema realmente grave.,4.4 Hay una disminución de la superficie forestal y de áreas naturales en el país.,4.5 El planeta está tan contaminado por productos químicos que ya supone un problema para la salud.,4.6 Conozco los riesgos que representa para la vida humana la desaparición de especies animales y vegetales.,4.7 Me preocupa lo que sucede con la tala de árboles.,Comentarios medioambiente,Tipo de discapacidad,¿Te reconoces como una persona con algún tipo de discapacidad?,Conoce GreenTIC,Instrumento
21085,D0,Sí,65,2,Omar Joel Hurtado Alegria,01/06,65060209,9,0,Masculino,...,Totalmente de acuerdo,Neutro,Totalmente de acuerdo,De acuerdo,Totalmente de acuerdo,Sin respuesta,NaN,No,No,Encuesta estudiantes
21086,D1,Sí,65,2,Edinson Sinisterra Urbano,01/06,65060222,22,12,Masculino,...,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Sin respuesta,NaN,No,No,Encuesta estudiantes
21087,D2,Sí,65,2,Jositn Santiago Suarez Cuero,01/06,65060223,23,10,Masculino,...,Neutro,Totalmente de acuerdo,Totalmente de acuerdo,Neutro,Neutro,Sin respuesta,NaN,No,No,Encuesta estudiantes
21088,D3,Sí,65,2,Velncia Tejada Cristian,01/06,65060227,27,11,Masculino,...,De acuerdo,En desacuerdo,Totalmente de acuerdo,De acuerdo,Neutro,Sin respuesta,NaN,No,No,Encuesta estudiantes
21089,D4,Sí,65,2,Joel Sneider Alomia Cuero,01/06,65060202,2,11,Masculino,...,Totalmente de acuerdo,De acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,De acuerdo,Sin respuesta,NaN,No,No,Encuesta estudiantes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21177,D93,Sí,65,5,Dennis Elian Valencia Bravo,01/06,65090525,25,16,Prefiero no decirlo,...,Totalmente de acuerdo,Neutro,Totalmente de acuerdo,Neutro,Totalmente de acuerdo,El medio ambiente me preocupa,NaN,No,No,Encuesta estudiantes
21178,D94,Sí,65,5,Marlon Stiwar Gamboa Moreno,01/06,65090507,7,16,Masculino,...,Totalmente de acuerdo,Neutro,Totalmente de acuerdo,Totalmente de acuerdo,Totalmente de acuerdo,Sin respuesta,NaN,No,Sí,Encuesta estudiantes
21179,D95,Sí,65,5,Juan Adrian Caicedo Caceres,01/06,65090504,4,0,Masculino,...,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Totalmente en desacuerdo,Sin respuesta,NaN,No,No,Encuesta estudiantes
21180,D96,Sí,65,5,Andres Felipe Arboleda,01/06,65090503,3,16,Masculino,...,En desacuerdo,Totalmente de acuerdo,De acuerdo,Totalmente de acuerdo,De acuerdo,Sin respuesta,NaN,No,No,Encuesta estudiantes


##### Pasar respuestas a números

In [5]:
respuestas_correctas = {
    "Un algoritmo es:":"Una secuencia lógica de pasos para realizar una tarea",
    "¿Para qué sirven los algoritmos?":"Para planificar la solución a un problema",
    "Un bucle es:":"Un conjunto de instrucciones que se ejecuta mientras se cumpla una condición",
    "¿Cuál de las siguientes opciones sí le permite al robot completar la misión de fotografiar cada tortuga?":"a.",
    "¿Qué mensaje deseaba enviar la líder Wayuú?":"c. Nublado",
    "¿Cuál de los siguientes códigos permite que el robot complete su misión sembrando café?":"a.",
    "¿Cuál será la foto con más vistas?":"c) Julio",
    "Ayuda al robot verde a salir del laberinto":"b.",
    "Óscar lleva 2 loncheras a la escuela todos los días ¿Cuál de las siguientes afirmaciones es falsa?":"c) Si Óscar empaca Deditos para merendar, puede hacer Arroz de pollo para almorzar",
    "¿Cuál de las siguientes hamburguesas tiene los ingredientes A, E y F?":"a.",
    "¿Qué botella debe cambiarse de color para que el resultado final sea una botella de color blanco?":"a) La botella B debe ser verde"}

de_acuerdo = {"Totalmente en desacuerdo":1,"En desacuerdo":2, "Neutro":3,
"De acuerdo":4,"Totalmente de acuerdo":5}

interesa_carreras = {"No la conozco":0,"La evitaría":1,"Me interesa poco":2,"Está entre mis preferidas":3}

estereotipo_h = {'a) Seguramente un hombre':2, 'b) Quizás un hombre':1,
                 'c) Creo que puede ser un hombre o una mujer':0,
                 'd) Quizás una mujer': -1, 'e) Seguramente una mujer':-2}

estereotipo_m = {'a) Seguramente un hombre':-2, 'b) Quizás un hombre':-1,
                 'c) Creo que puede ser un hombre o una mujer':0,
                 'd) Quizás una mujer': 1, 'e) Seguramente una mujer':2}
#The scores range from -2= ‘counter- stereotypic answer’ to 2= ‘stereotype congruent answer’ for all items.
genero_estereotipos = {'5.1 ¿Quién crees que ganará el concurso de matemáticas?':estereotipo_h,
                       '5.2 ¿Quién crees que es capitán del barco?':estereotipo_h,
                       '5.3 ¿Quién es la persona excluida de la construcción de la casa de madera?':estereotipo_m,
                       '5.4 ¿Quién crees que es el personaje que está sentado y esperando junto a la ventana?':estereotipo_m,
                       '5.5 ¿Quién es la persona que trabaja en educación?':estereotipo_m,
                       '5.6 ¿Quién crees que es la persona que prefiere estudiar ingeniería?':estereotipo_h}


In [6]:
col_index = ['Edad', 'Sexo', 'Código IE', '¿Te reconoces como una persona con algún tipo de discapacidad?']
col_caract = ['N registro','Edad', 'Sexo', 'Sector vivienda', 'Internet', 'Uso del dispositivo móvil',
              'Nivel escolaridad madre', 'Nivel escolaridad padre', 'Ocupación madre',
              'Ocupación padre', '¿Con quién vives?', 'Grado', 'Código IE',
              'Grupo', 'Conoce GreenTIC', 'Número de lista', '¿Te reconoces como una persona con algún tipo de discapacidad?']
#col_conocePC = est.filter(regex=r'^3.*', axis=1).columns.tolist()[0]
#col_PC = est.filter(regex=r'^3.*', axis=1).columns.tolist()[1:5]
col_autoeficacia = est.filter(regex=r'^3.*', axis=1).columns.tolist()
col_carreras = est.filter(regex=r'^1.', axis=1).columns.tolist()
col_interes = est.filter(regex=r'^2.1|2.2', axis=1).columns.tolist()
col_conocimiento = list(respuestas_correctas.keys())
col_ambiental = est.filter(regex=r'^4.', axis=1).columns.tolist()
col_genero = est.filter(regex=r'^5.', axis=1).columns.tolist()
col_genero

['5.1 ¿Quién crees que ganará el concurso de matemáticas?',
 '5.2 ¿Quién crees que es capitán del barco?',
 '5.3 ¿Quién es la persona excluida de la construcción de la casa de madera?',
 '5.4 ¿Quién crees que es el personaje que está sentado y esperando junto a la ventana?',
 '5.5 ¿Quién es la persona que trabaja en educación?',
 '5.6 ¿Quién crees que es la persona que prefiere estudiar ingeniería?']

In [7]:
df_conocimientos = est[['N registro']+col_conocimiento].melt(id_vars='N registro', value_name='Respuesta estudiante', var_name='Pregunta')
df_conocimientos['Respuesta correcta'] = df_conocimientos['Pregunta'].replace(respuestas_correctas)
df_conocimientos['Puntaje conocimiento'] = 1*(df_conocimientos['Respuesta estudiante'] == df_conocimientos['Respuesta correcta'])
df_estudiantes = df_conocimientos.pivot_table(index='N registro', columns='Pregunta', values='Puntaje conocimiento').reset_index()



df_estudiantes= pd.merge(est[col_caract], df_estudiantes, on='N registro')
df_estudiantes

,N registro,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,Ocupación padre,...,Un algoritmo es:,Un bucle es:,"¿Cuál de las siguientes hamburguesas tiene los ingredientes A, E y F?",¿Cuál de las siguientes opciones sí le permite al robot completar la misión de fotografiar cada tortuga?,¿Cuál de los siguientes códigos permite que el robot complete su misión sembrando café?,¿Cuál será la foto con más vistas?,¿Para qué sirven los algoritmos?,¿Qué botella debe cambiarse de color para que el resultado final sea una botella de color blanco?,¿Qué mensaje deseaba enviar la líder Wayuú?,Óscar lleva 2 loncheras a la escuela todos los días ¿Cuál de las siguientes afirmaciones es falsa?
0,1,14,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,Trabaja,...,0,0,1,1,1,0,0,1,0,1
1,2,16,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,Trabaja,...,0,1,0,0,1,0,0,0,0,0
2,3,14,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,1,0,1,1,0,0,1,0,0,0
3,4,15,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,Trabaja,...,1,0,0,0,0,0,0,0,1,0
4,5,14,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,Trabaja,...,1,0,1,0,0,0,1,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22977,D2012,15,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,Técnico o tecnólogo,No sé,Trabaja,No sé,...,1,0,0,1,0,0,1,0,1,1
22978,D2013,15,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,Secundaría,Técnico o tecnólogo,Trabaja,Trabaja,...,0,0,1,0,1,0,0,0,1,0
22979,D2014,15,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Primaria,Trabaja,Trabaja,...,0,0,0,0,0,0,0,0,0,1
22980,D2015,15,Masculino,Urbana,No,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,0,1,0,1,0,0,1,0,0,1


In [32]:
# carreras_df = est[['N registro'] + col_carreras].replace(interesa_carreras)
# df_estudiantes = pd.merge(df_estudiantes, carreras_df, on='N registro')
#
# genero_df = est[['N registro']+col_genero].replace(genero_estereotipos)
# df_estudiantes = pd.merge(df_estudiantes, genero_df, on='N registro')

df_ambiente = est[['N registro']+col_ambiental].replace(de_acuerdo)



In [33]:
df_estudiantes = pd.merge(df_estudiantes, df_ambiente, on='N registro')


In [34]:

df_estudiantes = pd.merge(df_estudiantes, est[['N registro']+col_autoeficacia], on='N registro')

df_estudiantes

,N registro,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,Ocupación padre,...,3.1 Siento que soy capaz de explicar lo que es el pensamiento computacional,3.2 Siento que puedo enumerar las sub-habilidades que componen el pensamiento computacional,3.3 Siento que soy capaz de dar ejemplos para explicar las sub-habilidades del pensamiento computacional,3.4 Siento que puedo explicar la forma en que las sub-habilidades del pensamiento computacional se correlacionan con la programación,3.5 Siento que puedo analizar un ejercicio y determinar qué sub-habilidades de pensamiento computacional busca desarrollar,3.6 Siento que puedo resolver problemas a través de programación,3.7 Siento que puedo implementar algoritmos,3.8 Siento que puedo crear un programa de computador,3.9 Siento que puedo automatizar tareas a través de la programación,3.10 Siento que puedo utilizar la computación para resolver problemas simples
0,1,14,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,Trabaja,...,3,1,2,2,3,3,2,1,2,3
1,2,16,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,Trabaja,...,5,4,5,5,5,5,5,5,5,5
2,3,14,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,1,1,2,1,1,1,3,2,2,2
3,4,15,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,Trabaja,...,2,3,2,3,4,4,4,1,3,5
4,5,14,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,Trabaja,...,3,3,3,3,3,3,4,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22977,D2012,15,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,Técnico o tecnólogo,No sé,Trabaja,No sé,...,3,3,3,4,5,5,5,5,5,5
22978,D2013,15,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,Secundaría,Técnico o tecnólogo,Trabaja,Trabaja,...,3,3,4,3,3,4,2,2,4,3
22979,D2014,15,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Primaria,Trabaja,Trabaja,...,4,4,3,4,5,5,4,4,5,4
22980,D2015,15,Masculino,Urbana,No,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,3,2,3,4,5,3,5,5,4,5


#### Calcular escalas

In [35]:
escalas_dict = {'medioambiente':
                    {'cols':list(est.filter(regex='^4.*').columns),
                     'cargas':[0.620,0.648,0.732,0.638,0.705,0.707,0.670]},
                'autoeficaciaPC':
                    {'cols':list(est.filter(regex='^3.1 |3.[2-5]|^3.7').columns),
                'cargas':[0.724,0.822,0.782,0.745,0.574,0.390]},
                'autoeficaciaProg':{
                    'cols':list(est.filter(regex='^3.6 |3.[8-9]|3.10').columns),
                    'cargas':[0.637,0.490,0.755,0.753]
                }}

In [37]:
df_estudiantes[col_ambiental+col_autoeficacia] = df_estudiantes[col_ambiental+col_autoeficacia].apply(pd.to_numeric, errors='coerce')


In [38]:
df_estudiantes = df_estudiantes.dropna(subset=col_ambiental+col_autoeficacia)
df_estudiantes

,N registro,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,Ocupación padre,...,3.1 Siento que soy capaz de explicar lo que es el pensamiento computacional,3.2 Siento que puedo enumerar las sub-habilidades que componen el pensamiento computacional,3.3 Siento que soy capaz de dar ejemplos para explicar las sub-habilidades del pensamiento computacional,3.4 Siento que puedo explicar la forma en que las sub-habilidades del pensamiento computacional se correlacionan con la programación,3.5 Siento que puedo analizar un ejercicio y determinar qué sub-habilidades de pensamiento computacional busca desarrollar,3.6 Siento que puedo resolver problemas a través de programación,3.7 Siento que puedo implementar algoritmos,3.8 Siento que puedo crear un programa de computador,3.9 Siento que puedo automatizar tareas a través de la programación,3.10 Siento que puedo utilizar la computación para resolver problemas simples
0,1,14,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,Trabaja,...,3,1,2,2,3,3,2,1,2,3
1,2,16,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,Trabaja,...,5,4,5,5,5,5,5,5,5,5
2,3,14,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,1,1,2,1,1,1,3,2,2,2
3,4,15,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,Trabaja,...,2,3,2,3,4,4,4,1,3,5
4,5,14,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,Trabaja,...,3,3,3,3,3,3,4,3,3,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22977,D2012,15,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,Técnico o tecnólogo,No sé,Trabaja,No sé,...,3,3,3,4,5,5,5,5,5,5
22978,D2013,15,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,Secundaría,Técnico o tecnólogo,Trabaja,Trabaja,...,3,3,4,3,3,4,2,2,4,3
22979,D2014,15,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Primaria,Trabaja,Trabaja,...,4,4,3,4,5,5,4,4,5,4
22980,D2015,15,Masculino,Urbana,No,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,3,2,3,4,5,3,5,5,4,5


In [39]:
for k in list(escalas_dict.keys()):
    try:
        df_estudiantes[k] = 100*df_estudiantes[escalas_dict[k]['cols']].add(-1).multiply(escalas_dict[k]['cargas']).sum(axis=1)/(4*sum(escalas_dict[k]['cargas']))
    except:
        print('Fallo en ', k)

In [40]:
resultados = df_estudiantes[col_conocimiento].sum(axis=1)
media = resultados.mean()
desv = resultados.std()

df_estudiantes['conocimiento'] = 50+(10*(resultados - media)/desv)



In [60]:
df_estudiantes

,N registro,Edad,Sexo,Sector vivienda,Internet,Uso del dispositivo móvil,Nivel escolaridad madre,Nivel escolaridad padre,Ocupación madre,Ocupación padre,...,3.5 Siento que puedo analizar un ejercicio y determinar qué sub-habilidades de pensamiento computacional busca desarrollar,3.6 Siento que puedo resolver problemas a través de programación,3.7 Siento que puedo implementar algoritmos,3.8 Siento que puedo crear un programa de computador,3.9 Siento que puedo automatizar tareas a través de la programación,3.10 Siento que puedo utilizar la computación para resolver problemas simples,medioambiente,autoeficaciaPC,autoeficaciaProg,conocimiento
0,1,14,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Técnico o tecnólogo,Titulo universitario,Trabaja,Trabaja,...,3,3,2,1,2,3,57.621822,27.947733,33.538899,58.174595
1,2,16,Masculino,Urbana,Sí,Uso propio (solo lo usas tú),Titulo universitario,Técnico o tecnólogo,Trabaja,Trabaja,...,5,5,5,5,5,5,75.000000,94.909586,100.000000,43.807896
2,3,14,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,1,1,3,2,2,2,79.713983,9.673025,18.956357,53.385695
3,4,15,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,No sé,No sé,Trabaja,Trabaja,...,4,4,4,1,3,5,92.521186,46.643547,61.034156,43.807896
4,5,14,Femenino,Urbana,Sí,Uso compartido con otras personas,Titulo universitario,Secundaría,Trabaja,Trabaja,...,3,3,4,3,3,3,71.461864,52.415160,50.000000,53.385695
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22977,D2012,15,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,Técnico o tecnólogo,No sé,Trabaja,No sé,...,5,5,5,5,5,5,100.000000,66.553134,100.000000,58.174595
22978,D2013,15,Masculino,Urbana,No,No tengo acceso a ningún dispositivo electrónico,Secundaría,Técnico o tecnólogo,Trabaja,Trabaja,...,3,4,2,2,4,3,69.184322,52.427545,58.557875,48.596795
22979,D2014,15,Femenino,Urbana,Sí,Uso compartido con otras personas,Secundaría,Primaria,Trabaja,Trabaja,...,5,5,4,4,5,4,93.241525,73.711915,88.206831,43.807896
22980,D2015,15,Masculino,Urbana,No,Uso compartido con otras personas,Secundaría,Secundaría,Trabaja,Trabaja,...,5,3,5,5,4,5,56.281780,61.462720,80.749526,53.385695


In [42]:
estudiantes = pd.merge(est, df_estudiantes[['N registro', 'conocimiento', 'autoeficaciaPC', 'autoeficaciaProg', 'medioambiente']])

In [43]:
df_interes = est[['N registro']+col_interes].melt(id_vars='N registro', value_name='Seleccion', var_name='Pregunta')
df_interes['Interes en tecnologia'] = 1*(df_interes['Seleccion'].str.contains('Tecnología'))
df_puntaje_interes = df_interes.pivot_table(index='N registro', values=['Interes en tecnologia'], aggfunc=['sum', 'max']).reset_index()
df_puntaje_interes.columns = ['N registro', 'Puntaje interes', 'Interesado en tecnologia']
df_puntaje_interes['Puntaje interes'] = 100*(df_puntaje_interes['Puntaje interes']/2)
#df_puntaje_interes = (100*df_interes.groupby('index')['Interes en tecnologia'].sum()/4).reset_index()
df_puntaje_interes

,N registro,Puntaje interes,Interesado en tecnologia
0,1,100.0,1
1,2,50.0,1
2,3,50.0,1
3,4,100.0,1
4,5,0.0,0
...,...,...,...
22977,D995,0.0,0
22978,D996,0.0,0
22979,D997,50.0,1
22980,D998,0.0,0


In [44]:
df_7 = pd.merge(estudiantes, df_puntaje_interes, on='N registro')
df_7

,N registro,Deseo participar en el estudio,Código IE,Grupo,Nombre,Fecha,ID,Número de lista,Edad,Sexo,...,Tipo de discapacidad,¿Te reconoces como una persona con algún tipo de discapacidad?,Conoce GreenTIC,Instrumento,conocimiento,autoeficaciaPC,autoeficaciaProg,medioambiente,Puntaje interes,Interesado en tecnologia
0,1,Sí,166,2,NaN,19/04,166090203,3,14,Masculino,...,NaN,No,No,Encuesta estudiantes,58.174595,27.947733,33.538899,57.621822,100.0,1
1,2,Sí,166,2,NaN,19/04,166090207,7,16,Masculino,...,NaN,No,No,Encuesta estudiantes,43.807896,94.909586,100.000000,75.000000,50.0,1
2,3,Sí,166,2,NaN,19/04,166090230,30,14,Femenino,...,NaN,No,No,Encuesta estudiantes,53.385695,9.673025,18.956357,79.713983,50.0,1
3,4,Sí,166,2,NaN,19/04,166090206,6,15,Masculino,...,NaN,No,No,Encuesta estudiantes,43.807896,46.643547,61.034156,92.521186,100.0,1
4,5,Sí,166,2,NaN,19/04,166090210,10,14,Femenino,...,NaN,No,No,Encuesta estudiantes,53.385695,52.415160,50.000000,71.461864,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22977,D2012,Sí,134,6,JEAN CARLOS RUIZ MOSCOTE,01/06,134080627,27,15,Masculino,...,NaN,No,No,Encuesta estudiantes,58.174595,66.553134,100.000000,100.000000,0.0,0
22978,D2013,Sí,134,6,JHONSAN DAVID SARMIENTO HERNANDEZ,01/06,134080628,28,15,Masculino,...,NaN,No,No,Encuesta estudiantes,48.596795,52.427545,58.557875,69.184322,0.0,0
22979,D2014,Sí,134,6,NAYBETH ADRIANA TOVAR GONZALEZ,01/06,134080629,29,15,Femenino,...,NaN,No,No,Encuesta estudiantes,43.807896,73.711915,88.206831,93.241525,0.0,0
22980,D2015,Sí,134,6,ESTEBAN DAVID VALIENTE CARABALLO,01/06,134080630,30,15,Masculino,...,NaN,No,No,Encuesta estudiantes,53.385695,61.462720,80.749526,56.281780,0.0,0


In [46]:
df_7['autoeficaciaProg'].mean()

61.925579659157385

In [47]:
df_7['Media autoeficaciaProg'] = df_7['autoeficaciaProg'].mean()
df_7['Std autoeficaciaProg'] = df_7['autoeficaciaProg'].std()
df_7['Media autoeficaciaPC'] = df_7['autoeficaciaPC'].mean()
df_7['Std autoeficaciaPC'] = df_7['autoeficaciaPC'].std()

In [48]:
df_7['Media nacional interés'] = df_7['Puntaje interes'].mean()
df_7['Std interes'] = df_7['Puntaje interes'].std()

### Calificar Dimensión

In [49]:
cols_dimension7 = ['conocimiento', 'Media autoeficaciaProg','Puntaje interes', 'Interesado en tecnologia', 'Media autoeficaciaPC', 'autoeficaciaPC', 'autoeficaciaProg', 'Media nacional interés','Std interes','Std autoeficaciaPC','Std autoeficaciaProg', 'N registro']
agrupadores_dimension7 = ['mean', 'max', 'mean', 'sum', 'max', 'mean','mean', 'max','max','max','max', 'count']
agg_funcs = dict(zip(cols_dimension7, agrupadores_dimension7))

In [50]:
agg_funcs

{'conocimiento': 'mean',
 'Media autoeficaciaProg': 'max',
 'Puntaje interes': 'mean',
 'Interesado en tecnologia': 'sum',
 'Media autoeficaciaPC': 'max',
 'autoeficaciaPC': 'mean',
 'autoeficaciaProg': 'mean',
 'Media nacional interés': 'max',
 'Std interes': 'max',
 'Std autoeficaciaPC': 'max',
 'Std autoeficaciaProg': 'max',
 'N registro': 'count'}

In [51]:
df_institucion = df_7.pivot_table(index=['Código IE'], values=cols_dimension7, aggfunc=agg_funcs).rename(columns={'N registro':'Cantidad estudiantes'})
df_institucion['% Interesado'] = (df_institucion['Interesado en tecnologia']/df_institucion['Cantidad estudiantes'])*100

In [52]:
## Cuántas desviaciones estándar respecto a la media
df_institucion['Diff autoeficaciaPC'] = (df_institucion['autoeficaciaPC'] - df_institucion['Media autoeficaciaPC'])/df_institucion['Std autoeficaciaPC']

df_institucion['Diff autoeficaciaProg'] = (df_institucion['autoeficaciaProg'] - df_institucion['Media autoeficaciaProg'])/df_institucion['Std autoeficaciaProg']

df_institucion['Diferencia interes'] = (df_institucion['Puntaje interes'] - df_institucion['Media nacional interés'])/df_institucion['Std interes']

In [53]:
df_institucion['Media autoeficaciaProg']

Código IE
1      61.92558
2      61.92558
3      61.92558
4      61.92558
5      61.92558
         ...   
248    61.92558
249    61.92558
250    61.92558
251    61.92558
252    61.92558
Name: Media autoeficaciaProg, Length: 252, dtype: float64

In [54]:
df_institucion['Nivel autoeficaciaPC'] =  pd.cut(df_institucion['autoeficaciaPC'], bins=[0, 20, 30, 50, 60, 80,100], labels=['1A', '1B', '2A', '2B','3', '5'], include_lowest=True)

df_institucion['Nivel autoeficaciaProg'] =  pd.cut(df_institucion['autoeficaciaProg'], bins=[0, 20, 30, 50, 60, 100], labels=['1A', '1B', '2A', '2B', '5'], include_lowest=True)

df_institucion['Nivel conocimiento'] =  pd.cut(df_institucion['conocimiento'], bins=[0, 30, 40, 50, 60, 100], labels=['1A', '1B', '2A', '2B', '5'], include_lowest=True)

df_institucion['Nivel Interes'] = pd.cut(df_institucion['% Interesado'], bins=[0, 20, 40, 60, 80, 100], labels=['1A', '1B', "2A", "2B", '5'], include_lowest=True)

df_institucion['Nivel Interes pares'] = pd.cut(df_institucion['Diferencia interes'], bins=[-3,-1,0,1,2,3], labels=['1B', "2A", "3", '4','5'], include_lowest=True)

df_institucion['Nivel AutoeficaciaPC pares'] = pd.cut(df_institucion['Diff autoeficaciaPC'], bins=[-3, 0, 1.5, 2, 4], labels=['2A',"2B", "3", '4'], include_lowest=True)

df_institucion['Nivel AutoeficaciaProg pares'] = pd.cut(df_institucion['Diff autoeficaciaProg'], bins=[-3, 0, 1.5, 2, 4], labels=['2A',"2B", "3", '4'], include_lowest=True)


df_institucion

,Interesado en tecnologia,Media autoeficaciaPC,Media autoeficaciaProg,Media nacional interés,Cantidad estudiantes,Puntaje interes,Std autoeficaciaPC,Std autoeficaciaProg,Std interes,autoeficaciaPC,...,Diff autoeficaciaPC,Diff autoeficaciaProg,Diferencia interes,Nivel autoeficaciaPC,Nivel autoeficaciaProg,Nivel conocimiento,Nivel Interes,Nivel Interes pares,Nivel AutoeficaciaPC pares,Nivel AutoeficaciaProg pares
Código IE,,,,,,,,,,,,,,,,,,,,,
1,60,44.458476,61.92558,32.323122,122,36.065574,26.84896,27.363284,37.932209,46.299039,...,0.068552,0.176151,0.098662,2A,5,2A,2A,3,2B,2B
2,27,44.458476,61.92558,32.323122,39,52.564103,26.84896,27.363284,37.932209,40.420184,...,-0.150408,-0.413376,0.533609,2A,2B,2A,2B,3,2A,2A
3,34,44.458476,61.92558,32.323122,59,39.830508,26.84896,27.363284,37.932209,49.334125,...,0.181595,0.267890,0.197916,2A,5,2A,2A,3,2B,2B
4,30,44.458476,61.92558,32.323122,46,51.086957,26.84896,27.363284,37.932209,47.185948,...,0.101586,0.052899,0.494668,2A,5,2B,2B,3,2B,2B
5,81,44.458476,61.92558,32.323122,105,56.666667,26.84896,27.363284,37.932209,50.755747,...,0.234544,0.438113,0.641764,2B,5,2B,2B,3,2B,2B
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,25,44.458476,61.92558,32.323122,56,28.571429,26.84896,27.363284,37.932209,32.815430,...,-0.433650,-0.271388,-0.098905,2A,2B,2A,2A,2A,2A,2A
249,50,44.458476,61.92558,32.323122,149,20.805369,26.84896,27.363284,37.932209,39.210915,...,-0.195447,-0.151464,-0.303640,2A,2B,2A,1B,2A,2A,2A
250,26,44.458476,61.92558,32.323122,82,19.512195,26.84896,27.363284,37.932209,40.925857,...,-0.131574,-0.139129,-0.337732,2A,2B,2A,1B,2A,2A,2A


In [55]:
df_institucion['Dimensión 7'] = df_institucion.filter(regex='^Nivel').min(axis=1)

In [59]:
df_institucion['Dimensión 7'].value_counts()

2A    143
1B     90
1A     10
2B      9
Name: Dimensión 7, dtype: int64

In [57]:
#df_dimension = df_institucion.filter(regex='^Nivel|Dim')
df_institucion.reset_index().to_excel('Dimensión 7.xlsx')